In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sherlock.db'
mask_file = '/data/eli/sherlock/movie_files/wholebrain.nii.gz'

OVERWRITE = True

MOVIE_RUN_LENGTHS = [946, 1030]
MOVIE_1_SCENES = '''Scene Onset Offset
     1     1    26
     2    27    35
     3    36    55
     4    56    86
     5    88   109
     6   110   130
     7   132   143
     8   145   157
     9   158   172
    10   174   192
    11   194   203
    12   206   225
    13   227   313
    14   315   361
    15   364   399
    16   400   503
    17   508   525
    18   527   534
    19   536   615
    20   616   633
    21   635   641
    22   643   679
    23   680   697
    24   700   747
    25   749   868
    26   873   891
    27   892   945
'''

MOVIE_2_SCENES = '''Scene Onset Offset
1           1          25
2          27          69
3          71         129
4         131         145
5         146         193
6         195         275
7         276         325
8         344         412
9         414         489
10         491         523
11         525         635
12         636         671
13         672         728
14         730         747
15         748         756
16         758         790
17         791         813
18         815         859
19         860         875
20         877         912
21         914         960
22         962        1000
23        1001        1030
'''

In [6]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file)

if not existed or OVERWRITE:
    data_files = glob.glob('/data/eli/sherlock/movie_files/*.nii')
    regex = re.compile("sherlock_movie_s([0-9]{1,2}).nii")
    for data_file in data_files:
        m = regex.search(data_file)
        subject = int(m.groups()[0])

        tsv_reader = csv.DictReader(MOVIE_1_SCENES.splitlines(), delimiter=' ', skipinitialspace=True)
        scenes = [row for row in tsv_reader]
        start_block = niidb.FMriActivationBlock(zscore=False)
        start_block.task = 'sherlock_movie1_rest'
        start_block.filename = data_file
        start_block.subject = subject
        start_block.run = 1
        start_block.start_time = 0
        start_block.end_time = int(scenes[0]['Onset']) - 1
        db.upsert(start_block)
        for (scene, onset, offset) in [s.values() for s in scenes]:
            block = niidb.FMriActivationBlock(zscore=False)
            block.filename = data_file
            block.subject = subject
            block.run = 1
            block.task = 'sherlock_movie1_scene%d' % int(scene)
            block.start_time = int(onset)
            block.end_time = int(offset)
            db.upsert(block)
            logging.info("Scene %d of subject %d's first run", int(scene), subject)
        
        tsv_reader = csv.DictReader(MOVIE_2_SCENES.splitlines(), delimiter=' ', skipinitialspace=True)
        scenes = [row for row in tsv_reader]
        start_block = niidb.FMriActivationBlock(zscore=False)
        start_block.task = 'sherlock_movie2_rest'
        start_block.filename = data_file
        start_block.subject = subject
        start_block.run = 2
        start_block.start_time = 0
        start_block.end_time = int(scenes[0]['Onset']) - 1
        for (scene, onset, offset) in [s.values() for s in scenes]:
            block = niidb.FMriActivationBlock(zscore=False)
            block.filename = data_file
            block.subject = subject
            block.run = 2
            block.task = 'sherlock_movie2_scene%d' % int(scene)
            block.start_time = int(onset)
            block.end_time = int(offset)
            db.upsert(block)
            logging.info("Scene %d of subject %d's second run", int(scene), subject)


05/13/2018 22:07:02 Scene 1 of subject 10's first run
05/13/2018 22:07:02 Scene 2 of subject 10's first run
05/13/2018 22:07:02 Scene 3 of subject 10's first run
05/13/2018 22:07:02 Scene 4 of subject 10's first run
05/13/2018 22:07:02 Scene 5 of subject 10's first run
05/13/2018 22:07:02 Scene 6 of subject 10's first run
05/13/2018 22:07:02 Scene 7 of subject 10's first run
05/13/2018 22:07:02 Scene 8 of subject 10's first run
05/13/2018 22:07:02 Scene 9 of subject 10's first run
05/13/2018 22:07:02 Scene 10 of subject 10's first run
05/13/2018 22:07:02 Scene 11 of subject 10's first run
05/13/2018 22:07:02 Scene 12 of subject 10's first run
05/13/2018 22:07:02 Scene 13 of subject 10's first run
05/13/2018 22:07:02 Scene 14 of subject 10's first run
05/13/2018 22:07:02 Scene 15 of subject 10's first run
05/13/2018 22:07:02 Scene 16 of subject 10's first run
05/13/2018 22:07:02 Scene 17 of subject 10's first run
05/13/2018 22:07:02 Scene 18 of subject 10's first run
05/13/2018 22:07:02

In [7]:
logging.info('Finished building NiiDb out of Sherlock_Merlin dataset')

05/13/2018 22:07:08 Finished building NiiDb out of Sherlock_Merlin dataset
